In [ ]:
!pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index
!pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
import efficientnet.tfkeras as efn
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from pickle import load,dump
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans

import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import gc

from sklearn.metrics import roc_auc_score

import json

from tensorflow.keras import layers,regularizers,Sequential,backend,callbacks,optimizers,metrics,losses
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow_addons as tfa

from tqdm.notebook import tqdm
import math

from pytorch_tabnet.tab_model import TabNetRegressor

# Model1


In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
df = pd.read_csv('../input/lish-moa/sample_submission.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
tns=train_targets_nonscored.iloc[:,1:]
tns_col=list(tns.loc[:,tns.sum()>3].columns)
tns_col=["sig_id"]+tns_col

train_targets_nonscored=train_targets_nonscored[tns_col]
# train_targets_nonscored

In [ ]:
train_features2=train_features.copy()
test_features2=test_features.copy()

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

In [ ]:
n_comp = 600  #<--Update
pca_g = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
#gpca= (pca_g.fit(data[GENES]))
gpca = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/gpca.pkl","rb"))
train2= (gpca.transform(train_features[GENES]))
test2 = (gpca.transform(test_features[GENES]))

train_gpca = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_gpca = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train_gpca), axis=1)
test_features = pd.concat((test_features, test_gpca), axis=1)

#CELLS
n_comp = 50  #<--Update

pca_c = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
#cpca= (pca_c.fit(data[CELLS]))
cpca = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/cpca.pkl","rb"))
train2= (cpca.transform(train_features[CELLS]))
test2 = (cpca.transform(test_features[CELLS]))

train_cpca = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_cpca = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train_cpca), axis=1)
test_features = pd.concat((test_features, test_cpca), axis=1)

In [ ]:


c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
#mask = (train_features[c_n].var() >= 0.85).values
mask = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/mask.pkl","rb"))
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)

In [ ]:

def fe_cluster_genes(train, test, n_clusters_g = 22, SEED = 42):
    
    features_g = GENES
    #features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        #kmeans_genes = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        kmeans_genes = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/kmeans_genes.pkl","rb"))
        #dump(kmeans_genes, open('kmeans_genes.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
   # train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_genes(train_features2,test_features2)

In [ ]:
def fe_cluster_cells(train, test, n_clusters_c = 4, SEED = 42):
    
    #features_g = GENES
    features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'c', n_clusters = n_clusters_c):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
#         kmeans_cells = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_cells, open('kmeans_cells.pkl', 'wb'))
        kmeans_cells = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/kmeans_cells.pkl","rb"))
        train[f'clusters_{kind}'] = kmeans_cells.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_cells.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
   # train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_cells(train_features2,test_features2)

train_pca=pd.concat((train_gpca,train_cpca),axis=1)
test_pca=pd.concat((test_gpca,test_cpca),axis=1)

def fe_cluster_pca(train, test,n_clusters=5,SEED = 42):
        data=pd.concat([train,test],axis=0)
#         kmeans_pca = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_pca, open('kmeans_pca.pkl', 'wb'))
        kmeans_pca = pickle.load(open("../input/moa-pretained-non-scored-targets-as-meta-features/kmeans_pca.pkl","rb"))
        train[f'clusters_pca'] = kmeans_pca.predict(train.values)
        test[f'clusters_pca'] = kmeans_pca.predict(test.values)
        train = pd.get_dummies(train, columns = [f'clusters_pca'])
        test = pd.get_dummies(test, columns = [f'clusters_pca'])
        return train, test
train_cluster_pca ,test_cluster_pca = fe_cluster_pca(train_pca,test_pca)

train_cluster_pca = train_cluster_pca.iloc[:,650:]
test_cluster_pca = test_cluster_pca.iloc[:,650:]

train_features_cluster=train_features2.iloc[:,876:]
test_features_cluster=test_features2.iloc[:,876:]


In [ ]:
gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [ ]:
def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
        df['c52_c42'] = df['c-52'] * df['c-42']
        df['c13_c73'] = df['c-13'] * df['c-73']
        df['c26_c13'] = df['c-23'] * df['c-13']
        df['c33_c6'] = df['c-33'] * df['c-6']
        df['c11_c55'] = df['c-11'] * df['c-55']
        df['c38_c63'] = df['c-38'] * df['c-63']
        df['c38_c94'] = df['c-38'] * df['c-94']
        df['c13_c94'] = df['c-13'] * df['c-94']
        df['c4_c52'] = df['c-4'] * df['c-52']
        df['c4_c42'] = df['c-4'] * df['c-42']
        df['c13_c38'] = df['c-13'] * df['c-38']
        df['c55_c2'] = df['c-55'] * df['c-2']
        df['c55_c4'] = df['c-55'] * df['c-4']
        df['c4_c13'] = df['c-4'] * df['c-13']
        df['c82_c42'] = df['c-82'] * df['c-42']
        df['c66_c42'] = df['c-66'] * df['c-42']
        df['c6_c38'] = df['c-6'] * df['c-38']
        df['c2_c13'] = df['c-2'] * df['c-13']
        df['c62_c42'] = df['c-62'] * df['c-42']
        df['c90_c55'] = df['c-90'] * df['c-55']
        
        
        for feature in features_c:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
        for feature in gsquarecols:
            df[f'{feature}_squared'] = df[feature] ** 2        
        
    return train, test

train_features2,test_features2=fe_stats(train_features2,test_features2)

In [ ]:
train_features_stats=train_features2.iloc[:,902:]
test_features_stats=test_features2.iloc[:,902:]

train_features = pd.concat((train_features, train_features_cluster,train_cluster_pca,train_features_stats), axis=1)
test_features = pd.concat((test_features, test_features_cluster,test_cluster_pca,test_features_stats), axis=1)

train = train_features.merge(train_targets_nonscored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_nonscored.columns]


train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)



In [ ]:
train_model1=train.copy()
test_model1=test.copy()

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])

feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   

In [ ]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:


class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [ ]:

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/moa-pretained-non-scored-targets-as-meta-features/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    
#     model.to(DEVICE)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
# #         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
# #             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_nonscored.pth")
        
# #         elif(EARLY_STOP == True):
            
# #             early_step += 1
# #             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    model.load_state_dict(torch.load(f"../input/moa-pretained-non-scored-targets-as-meta-features/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
#     return oof, predictions
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

        
    return predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print(seed)
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)


test_[target_cols] = predictions

In [ ]:
# train[target_cols]

In [ ]:
train = train.merge(train_targets_scored, on='sig_id')
target = train[train_targets_scored.columns]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [ ]:
feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id','kfold']]

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/moa-pretained-non-scored-targets-as-meta-features/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
#     model.to(DEVICE)

#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
# #         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_scored.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"../input/moa-pretained-non-scored-targets-as-meta-features/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
   # return oof, predictions
    return  predictions

In [ ]:
# def run_k_fold(NFOLDS, seed):
#     oof = np.zeros((len(train), len(target_cols)))
#     predictions = np.zeros((len(test), len(target_cols)))
    
#     for fold in range(NFOLDS):
#         oof_, pred_ = run_training(fold, seed)
        
#         predictions += pred_ / NFOLDS
#         oof += oof_
        
#     return oof, predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

        
    return predictions

In [ ]:
# # Averaging on multiple SEEDS

# SEED = [0,1,2]  #<-- Update
# oof = np.zeros((len(train), len(target_cols)))
# predictions = np.zeros((len(test), len(target_cols)))

# for seed in SEED:
    
#     oof_, predictions_ = run_k_fold(NFOLDS, seed)
#     oof += oof_ / len(SEED)
#     predictions += predictions_ / len(SEED)

# train[target_cols] = oof
# test[target_cols] = predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print(seed)
    
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)

test[target_cols] = predictions

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
# sub

In [ ]:
tmp1=pd.read_csv("../input/moa-pretained-non-scored-targets-as-meta-features/submission.csv")
# tmp1

In [ ]:
#(abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #3.07895243e-06

In [ ]:

ss_model1=sub

In [ ]:
gc.collect()

# Model2

In [ ]:
train = train_model1.copy()
test = test_model1.copy()
target = train[train_targets_nonscored.columns]

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])

feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.2)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))  
        
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-3rd-layers/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    
#     model.to(DEVICE)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
#         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_nonscored.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-3rd-layers/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return  predictions

In [ ]:
def run_k_fold(NFOLDS, seed):
    
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
    
        
    return predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print(seed)
    predictions_ = run_k_fold(NFOLDS, seed)
    
    predictions += predictions_ / len(SEED)

test_[target_cols] = predictions

In [ ]:
train = train.merge(train_targets_scored, on='sig_id')
target = train[train_targets_scored.columns]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id','kfold']]

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-3rd-layers/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
#     model.to(DEVICE)

#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
#         if valid_loss < best_loss:
            
#             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_scored.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-3rd-layers/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return  predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        
        
    return  predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)
    
    predictions += predictions_ / len(SEED)


test[target_cols] = predictions

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

In [ ]:

ss_model2=sub

In [ ]:
tmp1=pd.read_csv("../input/pretained-non-scored-targets-as-meta-3rd-layers/submission.csv")
# tmp1

In [ ]:
# (abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #3.842e-06

In [ ]:
gc.collect()

# Model3


In [ ]:
train = train_model1.copy()
test = test_model1.copy()
target = train[train_targets_nonscored.columns]

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])

feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.2)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm5 = nn.BatchNorm1d(hidden_size)
        self.dropout5 = nn.Dropout(0.2)
        self.dense5 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))  
        
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.relu(self.dense4(x))
        
        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        
        return x

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
# #     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    
#     model.to(DEVICE)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
#     for epoch in range(EPOCHS):
        
#         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
#         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
# #         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_nonscored.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    print(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_nonscored.pth")
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return  predictions

In [ ]:
def run_k_fold(NFOLDS, seed):
  
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
       
        
    return  predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)


test_[target_cols] = predictions

In [ ]:
train = train.merge(train_targets_scored, on='sig_id')
target = train[train_targets_scored.columns]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id','kfold']]

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
#     model.to(DEVICE)

#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
# #     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
# #     early_stopping_steps = EARLY_STOPPING_STEPS
# #     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
# #         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
# #             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_scored.pth")
        
# #         elif(EARLY_STOP == True):
            
# #             early_step += 1
# #             if (early_step >= early_stopping_steps):
# #                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_scored.pth","cpu"))
    print(f"../input/pretained-non-scored-targets-as-meta-4th-layers/SEED{seed}_FOLD{fold}_scored.pth")
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

        
    return  predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)
 
    predictions += predictions_ / len(SEED)

test[target_cols] = predictions

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

In [ ]:

ss_model3=sub

In [ ]:
# tmp1=pd.read_csv("../input/pretained-non-scored-targets-as-meta-4th-layers/submission.csv")
# tmp1

In [ ]:
# (abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #4.5585e-06

In [ ]:
gc.collect()

# Model4

In [ ]:
n_comp_GENES = 463
n_comp_CELLS = 60
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
#QT_n_quantile_min=50, 
#QT_n_quantile_max=1000,
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
def calc_QT_par_kurt(QT_n_quantile_min=10, QT_n_quantile_max=200):
    # Calculation parameters of function: n_quantile(kurtosis) = k1*kurtosis + k0
    # For Train & Test datasets (GENES + CELLS features): minimum kurtosis = 1.53655, maximum kurtosis = 30.4929
    
    a = np.array([[1.53655,1], [30.4929,1]])
    b = np.array([QT_n_quantile_min, QT_n_quantile_max])
    
    return np.linalg.solve(a, b)

def n_quantile_for_kurt(kurt, calc_QT_par_kurt_transform):
    # Calculation parameters of function: n_quantile(kurtosis) = calc_QT_par_kurt_transform[0]*kurtosis + calc_QT_par_kurt_transform[1]
    return int(calc_QT_par_kurt_transform[0]*kurt + calc_QT_par_kurt_transform[1])

In [ ]:

for col in (GENES + CELLS):

    #kurt = max(kurtosis(train_features[col]), kurtosis(test_features[col]))
    #QuantileTransformer_n_quantiles = n_quantile_for_kurt(kurt, calc_QT_par_kurt(QT_n_quantile_min, QT_n_quantile_max))
    #transformer = QuantileTransformer(n_quantiles=QuantileTransformer_n_quantiles,random_state=0, output_distribution="normal")
    
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")   # from optimal commit 9
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
# GENES

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])


gpca = pickle.load(open("../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/gpca.pkl","rb"))
data2 = (gpca.transform(data[GENES]))


train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)


# CELLS

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])


cpca = pickle.load(open("../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/cpca.pkl","rb"))
data2 = (cpca.transform(data[CELLS]))


train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features

In [ ]:
data = train_features.append(test_features)
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)

mask0 = pickle.load(open("../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/mask0.pkl","rb"))

train_features_transformed=train_features[train_features.columns[mask0]]

test_features_transformed=test_features[train_features.columns[mask0]]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
train = train
test = test_features
target = train[train_targets_scored.columns]

train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [ ]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=5)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)

In [ ]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct    
    
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds


class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1500

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(Dropout_Model)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(Dropout_Model)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.load_state_dict(torch.load(f"../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/SEED_{seed}_FOLD_{fold}.pth","cpu"))
#     model.to(DEVICE)
    
# #     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# #     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
# #                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
# #     early_stopping_steps = EARLY_STOPPING_STEPS
# #     early_step = 0
   
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"FOLD: {fold}, valid_loss: {valid_loss}")

# #         if valid_loss is not None and valid_loss < best_loss:
            
# #             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED_{seed}_FOLD_{fold}.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/SEED_{seed}_FOLD_{fold}.pth","cpu"))
    print(f"../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/SEED_{seed}_FOLD_{fold}.pth")
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

        
    return  predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0, 1, 2]

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)

test[target_cols] = predictions

In [ ]:
train1= pd.read_csv('../input/lish-moa/train_features.csv')
test1= pd.read_csv('../input/lish-moa/test_features.csv')


In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.loc[test1['cp_type']=="ctl_vehicle", target_cols] = 0

In [ ]:

ss_model4=sub

In [ ]:
# tmp1=pd.read_csv("../input/moa-pytorch-rankgauss-pca-nn-upgrade-3d-visual/submission.csv")

In [ ]:
# (abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #2.02e-06

In [ ]:
gc.collect()

# Model5

In [ ]:
# Import train data, drop sig_id, cp_type

train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
non_ctl_idx = train_features.loc[train_features['cp_type']!='ctl_vehicle'].index.to_list()
train_features = train_features.drop(['sig_id','cp_type','cp_dose','cp_time'],axis=1)
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_scored = train_targets_scored.drop('sig_id',axis=1)
labels_train = train_targets_scored.values

# Drop training data with ctl vehicle

train_features = train_features.iloc[non_ctl_idx]
labels_train = labels_train[non_ctl_idx]

# Import test data
ss = pd.read_csv('../input/lish-moa/sample_submission.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_features = test_features.drop(['sig_id','cp_dose','cp_time'],axis=1)

# Import predictors from public kernel

json_file_path = '../input/t-test-pca-rfe-logistic-regression/main_predictors.json'

with open(json_file_path, 'r') as j:
    predictors = json.loads(j.read())
    predictors = predictors['start_predictors']

In [ ]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(seed=53)

In [ ]:
cs = train_features.columns.str.contains('c-')
gs = train_features.columns.str.contains('g-')

def preprocessor(train,test):
    
    # PCA
    
    n_gs = 2 #2 # No of PCA comps to include
    n_cs = 50#100 # No of PCA comps to include
    
    pca_cs = PCA(n_components = n_cs)
    pca_gs = PCA(n_components = n_gs)

    train_pca_gs = pca_gs.fit_transform(train[:,gs])
    train_pca_cs = pca_cs.fit_transform(train[:,cs])
    test_pca_gs = pca_gs.transform(test[:,gs])
    test_pca_cs = pca_cs.transform(test[:,cs])
    
    # c-mean, g-mean
    
    train_c_mean = train[:,cs].mean(axis=1)
    test_c_mean = test[:,cs].mean(axis=1)
    train_c_std = train[:,cs].std(axis=1)
    test_c_std = test[:,cs].std(axis=1)
    train_g_mean = train[:,gs].mean(axis=1)
    test_g_mean = test[:,gs].mean(axis=1)
    train_g_std = train[:,gs].std(axis=1)
    test_g_std = test[:,gs].std(axis=1)
    
    # Append Features
    
    train = np.concatenate((train,train_pca_gs,train_pca_cs,train_c_mean[:,np.newaxis]
                            ,train_g_std[:,np.newaxis]),axis=1)
    test = np.concatenate((test,test_pca_gs,test_pca_cs,test_c_mean[:,np.newaxis],
                           test_g_std[:,np.newaxis]),axis=1)
    
    # Scaler for numerical values

    # Scale train data
    scaler = preprocessing.StandardScaler()

    train = scaler.fit_transform(train)

    # Scale Test data
    test = scaler.transform(test)
    
    return train, test

n_labels = train_targets_scored.shape[1]
n_train = train_features.shape[0]
n_test = test_features.shape[0]


# Prediction Clipping Thresholds

p_min = 0.0005
p_max = 0.9995

# OOF Evaluation Metric with clipping and no label smoothing

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))




In [ ]:
def build_model(n_features, n_features_2, n_labels, label_smoothing = 0.0005):    
    input_1 = layers.Input(shape = (n_features,), name = 'Input1')
    input_2 = layers.Input(shape = (n_features_2,), name = 'Input2')

    head_1 = Sequential([
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(512, activation="elu"), 
        layers.BatchNormalization(),
        layers.Dropout(0.1),
        layers.Dense(256, activation = "elu")
        ],name='Head1') 

    input_3 = head_1(input_1)
    input_3_concat = layers.Concatenate()([input_2, input_3])

    head_2 = Sequential([
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(512, "relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(512, "elu"),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(256, "relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.1),
        layers.Dense(256, "elu")
        ],name='Head2')

    input_4 = head_2(input_3_concat)
    input_4_avg = layers.Average()([input_3, input_4]) 

    head_3 = Sequential([
        layers.BatchNormalization(),
        layers.Dense(256, kernel_initializer='lecun_normal', activation='selu'),
        layers.BatchNormalization(),
        layers.Dense(n_labels, kernel_initializer='lecun_normal', activation='selu'),
        layers.BatchNormalization(),
        layers.Dense(n_labels, activation="sigmoid")
        ],name='Head3')

    output = head_3(input_4_avg)

    
    model = tf.keras.Model(inputs = [input_1, input_2], outputs = output)
    opt = tf.keras.optimizers.Adam(learning_rate=0.002)
    model.compile(opt, loss=losses.BinaryCrossentropy(label_smoothing=label_smoothing), metrics=logloss)
    
    return model

In [ ]:
train_targets=train_targets_scored.iloc[non_ctl_idx].reset_index(drop=True)
train_features=train_features.reset_index(drop=True)

In [ ]:
N_STARTS = 3
np.random.seed(1)
seeds = np.random.randint(0,100,size=N_STARTS)
res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in seeds:
    
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=5,random_state=seed,shuffle=True).split(train_features.values, train_targets)):
        print(f'Fold {n}')
        
        X_train0,data_test = preprocessor(train_features.values,
                           test_features.drop('cp_type',axis=1).values)
        
        X_train, X_test = X_train0[tr],X_train0[te]
#         _,data_test = preprocessor(train_features.iloc[tr].values,
#                                    test_features.drop('cp_type',axis=1).values)
        
        X_train_2 = train_features.iloc[tr][predictors].values
        X_test_2 = train_features.iloc[te][predictors].values
        data_test_2 = test_features[predictors].values
        y_train = labels_train[tr]
        y_test = labels_train[te]
        n_features = X_train.shape[1]
        n_features_2 = X_train_2.shape[1]
        
        model = build_model(n_features, n_features_2, n_labels)
        
#         reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.1, patience=2, mode='min', min_lr=1E-5)
#         early_stopping = callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=10, mode='min',restore_best_weights=True)

#         sv = tf.keras.callbacks.ModelCheckpoint(
#             'seed-%i-fold-%i-model.h5'%(seed,n), monitor='val_loss', verbose=0, save_best_only=True,
#             save_weights_only=True, mode='min', save_freq='epoch')
#         model.fit([X_train,X_train_2],y_train,shuffle=True,
#                   validation_data=([X_test,X_test_2],y_test),
#                   epochs=30, batch_size=128,
#                   callbacks=[sv,early_stopping,reduce_lr], verbose=1
#                  )
        model.load_weights(f"../input/multi-input-resnet-model/seed-{seed}-fold-{n}-model.h5")
        ss.loc[:, train_targets.columns] += model.predict([data_test,data_test_2])
        #res.loc[te, train_targets.columns] += model.predict([X_test,X_test_2])
        print(f"../input/multi-input-resnet-model/seed-{seed}-fold-{n}-model.h5")
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
#res.loc[:, train_targets.columns] /= N_STARTS

In [ ]:
train_targets_scored0 = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test1= pd.read_csv('../input/lish-moa/test_features.csv')

# tmp=train_targets_scored0.iloc[non_ctl_idx].reset_index(drop=True)
# res_sig_id=tmp.loc[:,["sig_id"]]

# res=pd.concat([res_sig_id,res],axis=1)

# valid_results = train_targets_scored0.drop(columns=train_targets.columns).merge(res, on='sig_id', how='left').fillna(0)


# valid_results

In [ ]:
ss.loc[test1['cp_type']=="ctl_vehicle", train_targets.columns] = 0


In [ ]:

ss_model5=ss

In [ ]:
# tmp1=pd.read_csv("../input/multi-input-resnet-model-inferce/submission.csv")

In [ ]:
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #3.5646e-07

In [ ]:
gc.collect()

# Model6


# Model7

# Model8

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
df = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
tns=train_targets_nonscored.iloc[:,1:]

tns_col=list(tns.loc[:,tns.sum()>3].columns)
tns_col=["sig_id"]+tns_col

train_targets_nonscored=train_targets_nonscored[tns_col]


train_features2=train_features.copy()
test_features2=test_features.copy()


GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

In [ ]:
n_comp = 600  #<--Update
pca_g = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
#gpca= (pca_g.fit(train_features[GENES]))
gpca = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/gpca.pkl","rb"))
train2= (gpca.transform(train_features[GENES]))
test2 = (gpca.transform(test_features[GENES]))

train_gpca = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_gpca = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train_gpca), axis=1)
test_features = pd.concat((test_features, test_gpca), axis=1)


#CELLS
n_comp = 50  #<--Update

pca_c = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
#cpca= (pca_c.fit(train_features[CELLS]))
cpca = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/cpca.pkl","rb"))
train2= (cpca.transform(train_features[CELLS]))
test2 = (cpca.transform(test_features[CELLS]))

train_cpca = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_cpca = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train_cpca), axis=1)
test_features = pd.concat((test_features, test_cpca), axis=1)

In [ ]:
c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
#mask = (train_features[c_n].var() >= 0.85).values
mask = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/mask.pkl","rb"))
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)

In [ ]:
def fe_cluster_genes(train, test, n_clusters_g = 22, SEED = 42):
    
    features_g = GENES
    #features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
#         kmeans_genes = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_genes, open('kmeans_genes.pkl', 'wb'))
        kmeans_genes = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/kmeans_genes.pkl","rb"))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
   # train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_genes(train_features2,test_features2)


def fe_cluster_cells(train, test, n_clusters_c = 4, SEED = 42):
    
    #features_g = GENES
    features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'c', n_clusters = n_clusters_c):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
#         kmeans_cells = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_cells, open('kmeans_cells.pkl', 'wb'))
        kmeans_cells = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/kmeans_cells.pkl","rb"))
        train[f'clusters_{kind}'] = kmeans_cells.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_cells.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
   # train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_cells(train_features2,test_features2)


train_pca=pd.concat((train_gpca,train_cpca),axis=1)
test_pca=pd.concat((test_gpca,test_cpca),axis=1)


def fe_cluster_pca(train, test,n_clusters=5,SEED = 42):
        data=pd.concat([train,test],axis=0)
#         kmeans_pca = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_pca, open('kmeans_pca.pkl', 'wb'))
        kmeans_pca = pickle.load(open("../input/pca-of-moa-pretained-non-scored-tabnet/kmeans_pca.pkl","rb"))
        train[f'clusters_pca'] = kmeans_pca.predict(train.values)
        test[f'clusters_pca'] = kmeans_pca.predict(test.values)
        train = pd.get_dummies(train, columns = [f'clusters_pca'])
        test = pd.get_dummies(test, columns = [f'clusters_pca'])
        return train, test
train_cluster_pca ,test_cluster_pca = fe_cluster_pca(train_pca,test_pca)


train_cluster_pca = train_cluster_pca.iloc[:,650:]
test_cluster_pca = test_cluster_pca.iloc[:,650:]


train_features_cluster=train_features2.iloc[:,876:]
test_features_cluster=test_features2.iloc[:,876:]


gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [ ]:
def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
        df['c52_c42'] = df['c-52'] * df['c-42']
        df['c13_c73'] = df['c-13'] * df['c-73']
        df['c26_c13'] = df['c-23'] * df['c-13']
        df['c33_c6'] = df['c-33'] * df['c-6']
        df['c11_c55'] = df['c-11'] * df['c-55']
        df['c38_c63'] = df['c-38'] * df['c-63']
        df['c38_c94'] = df['c-38'] * df['c-94']
        df['c13_c94'] = df['c-13'] * df['c-94']
        df['c4_c52'] = df['c-4'] * df['c-52']
        df['c4_c42'] = df['c-4'] * df['c-42']
        df['c13_c38'] = df['c-13'] * df['c-38']
        df['c55_c2'] = df['c-55'] * df['c-2']
        df['c55_c4'] = df['c-55'] * df['c-4']
        df['c4_c13'] = df['c-4'] * df['c-13']
        df['c82_c42'] = df['c-82'] * df['c-42']
        df['c66_c42'] = df['c-66'] * df['c-42']
        df['c6_c38'] = df['c-6'] * df['c-38']
        df['c2_c13'] = df['c-2'] * df['c-13']
        df['c62_c42'] = df['c-62'] * df['c-42']
        df['c90_c55'] = df['c-90'] * df['c-55']
        
        
        for feature in features_c:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
        for feature in gsquarecols:
            df[f'{feature}_squared'] = df[feature] ** 2        
        
    return train, test

train_features2,test_features2=fe_stats(train_features2,test_features2)


train_features_stats=train_features2.iloc[:,902:]
test_features_stats=test_features2.iloc[:,902:]

In [ ]:
train_features = pd.concat((train_features, train_features_cluster,train_cluster_pca,train_features_stats), axis=1)
test_features = pd.concat((test_features, test_features_cluster,test_cluster_pca,test_features_stats), axis=1)

train = train_features.merge(train_targets_nonscored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
#test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features
target = train[train_targets_nonscored.columns]


train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)


target_cols = target.drop('sig_id', axis=1).columns.values.tolist()


train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])


feature_cols = [c for c in train.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    
    
    
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   

In [ ]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss
    
    
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 26
BATCH_SIZE = 256
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
#     mskf = MultilabelStratifiedKFold(n_splits=5,random_state=seed)
#     for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#          train.loc[v_idx, 'kfold'] = int(f)
#     train['kfold'] = train['kfold'].astype(int)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )

#     model.load_state_dict(torch.load(f"../input/pca-of-moa-pretained-non-scored-tabnet/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
#     model.to(DEVICE)
    
# #     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# #     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
# #                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
# #     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
# #     early_stopping_steps = EARLY_STOPPING_STEPS
# #     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
#     best_loss = np.inf
    
# #     for epoch in range(EPOCHS):
        
# #         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
# #         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#     valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#     print(f"SEED: {seed} ,FOLD: {fold}, valid_loss: {valid_loss}")
        
#         if valid_loss < best_loss:
            
#             best_loss = valid_loss
#     oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED{seed}_FOLD{fold}_nonscored.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    model.load_state_dict(torch.load(f"../input/pca-of-moa-pretained-non-scored-tabnet/SEED{seed}_FOLD{fold}_nonscored.pth","cpu"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

        
    return predictions

In [ ]:
SEED = [0,1,2]  #<-- Update
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)


test_[target_cols] = predictions

In [ ]:
test=test_
test.drop(columns=["sig_id"], inplace=True)
train_targets_scored.drop(columns=["sig_id"], inplace=True)
X_test = test.values

In [ ]:
# test_

In [ ]:
train_targets_scored=train_targets_scored[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# train_targets_scored

In [ ]:

submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
ss=submission.copy()


In [ ]:
MAX_EPOCH=200
N_STARTS = 3

res.loc[:, train_targets_scored.columns] = 0
ss.loc[:, train_targets_scored.columns] = 0
NB_SPLITS = 5

scores_auc = []
for seed in range(N_STARTS):
    
    seed+=42
    #mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=(seed+10), shuffle=True)
    for fold_nb in range(NB_SPLITS):
        print("FOLDS : ", fold_nb)
        
#         ## model
#         X_train, y_train = train.values[train_idx, :], train_targets_scored.values[train_idx, :]
#         X_val, y_val = train.values[val_idx, :], train_targets_scored.values[val_idx, :]
# #         tabnet_params = dict(n_d=32, n_a=32, n_steps=1, gamma=1.3,
# #                      lambda_sparse=0, optimizer_fn=torch.optim.Adam,
# #                      optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
# #                      mask_type='entmax',
# #                      scheduler_params=dict(mode="min",
# #                                            patience=5,
# #                                            min_lr=1e-5,
# #                                            factor=0.9,),
# #                      scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
# #                      verbose=10,
# #                      seed=seed
# #                      )
# #         model = TabNetRegressor(**tabnet_params)

# #         model.fit(X_train=X_train,
# #                   y_train=y_train,
# #                   eval_set=[(X_val, y_val)],
# #                   eval_name = ["val"],
# #                   eval_metric = ["logits_ll"],
# #                   max_epochs=MAX_EPOCH,
# #                   patience=20, batch_size=1024, virtual_batch_size=128,
# #                   num_workers=1, drop_last=False,
# #                   # use binary cross entropy as this is not a regression problem
# #                   loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
# #         model.save_model("tabnet_model"+f"_fold_{fold_nb}_seed_{seed}")
        model = TabNetRegressor()
        model.load_model(f"../input/pca-of-moa-pretained-non-scored-tabnet/tabnet_model_fold_{fold_nb}_seed_{seed}.zip")
#         preds_val = model.predict(X_val)
#         # Apply sigmoid to the predictions
#         preds =  1 / (1 + np.exp(-preds_val))
# #         score = np.min(model.history["val_logits_ll"])

#         res.loc[val_idx, train_targets_scored.columns] += preds 
        


        # preds on test
        preds_test = model.predict(X_test)
        test_preds = (1 / (1 + np.exp(-preds_test)))
        ss.loc[:, train_targets_scored.columns] += test_preds 

# res.loc[:, train_targets_scored.columns] /= N_STARTS
ss.loc[:, train_targets_scored.columns] /= ((fold_nb+1) * N_STARTS)

In [ ]:
train_targets_scored0= pd.read_csv('../input/lish-moa/train_targets_scored.csv')
target_cols=train_targets_scored0.columns[1:]

In [ ]:
test1= pd.read_csv('../input/lish-moa/test_features.csv')
train1= pd.read_csv('../input/lish-moa/train_features.csv')

In [ ]:
ss.loc[test1['cp_type']=="ctl_vehicle", target_cols] = 0
# ss

In [ ]:
# tmp1=pd.read_csv("../input/pca-of-moa-pretained-non-scored-tabnet/submission.csv")

In [ ]:
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #5.2197e-06

In [ ]:
ss_model8=ss
gc.collect()

# Model9

In [ ]:
SEED=42
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(SEED)

train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')

submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
train_features2=train_features.copy()
test_features2=test_features.copy()

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
    
n_comp = 600  #<--Update
pca_g = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
#gpca= (pca_g.fit(data[GENES]))
gpca = pickle.load(open("../input/tabnetregressor-try-na-train-infe/gpca.pkl","rb"))
train2= (gpca.transform(train_features[GENES]))
test2 = (gpca.transform(test_features[GENES]))

train_gpca = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_gpca = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train_gpca), axis=1)
test_features = pd.concat((test_features, test_gpca), axis=1)


#CELLS
n_comp = 50  #<--Update

pca_c = PCA(n_components=n_comp, random_state=42)
data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
#cpca= (pca_c.fit(data[CELLS]))
cpca = pickle.load(open("../input/tabnetregressor-try-na-train-infe/cpca.pkl","rb"))
train2= (cpca.transform(train_features[CELLS]))
test2 = (cpca.transform(test_features[CELLS]))

train_cpca = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_cpca = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train_cpca), axis=1)
test_features = pd.concat((test_features, test_cpca), axis=1)



c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
#mask = (train_features[c_n].var() >= 0.85).values
mask = pickle.load(open("../input/tabnetregressor-try-na-train-infe/mask.pkl","rb"))
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)


def fe_cluster_genes(train, test, n_clusters_g = 22, SEED = 42):
    
    features_g = GENES
    #features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        #kmeans_genes = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        kmeans_genes = pickle.load(open("../input/tabnetregressor-try-na-train-infe/kmeans_genes.pkl","rb"))
        #dump(kmeans_genes, open('kmeans_genes.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
   # train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_genes(train_features2,test_features2)


def fe_cluster_cells(train, test, n_clusters_c = 4, SEED = 42):
    
    #features_g = GENES
    features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'c', n_clusters = n_clusters_c):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
#         kmeans_cells = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_cells, open('kmeans_cells.pkl', 'wb'))
        kmeans_cells = pickle.load(open("../input/tabnetregressor-try-na-train-infe/kmeans_cells.pkl","rb"))
        train[f'clusters_{kind}'] = kmeans_cells.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_cells.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
   # train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2 ,test_features2=fe_cluster_cells(train_features2,test_features2)

In [ ]:
train_pca=pd.concat((train_gpca,train_cpca),axis=1)
test_pca=pd.concat((test_gpca,test_cpca),axis=1)

def fe_cluster_pca(train, test,n_clusters=5,SEED = 42):
        data=pd.concat([train,test],axis=0)
#         kmeans_pca = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         dump(kmeans_pca, open('kmeans_pca.pkl', 'wb'))
        kmeans_pca = pickle.load(open("../input/tabnetregressor-try-na-train-infe/kmeans_pca.pkl","rb"))
        train[f'clusters_pca'] = kmeans_pca.predict(train.values)
        test[f'clusters_pca'] = kmeans_pca.predict(test.values)
        train = pd.get_dummies(train, columns = [f'clusters_pca'])
        test = pd.get_dummies(test, columns = [f'clusters_pca'])
        return train, test
train_cluster_pca ,test_cluster_pca = fe_cluster_pca(train_pca,test_pca)

train_cluster_pca = train_cluster_pca.iloc[:,650:]
test_cluster_pca = test_cluster_pca.iloc[:,650:]

train_features_cluster=train_features2.iloc[:,876:]
test_features_cluster=test_features2.iloc[:,876:]

gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
#         df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
#         df['g_kurt'] = df[features_g].kurtosis(axis = 1)
#         df['g_skew'] = df[features_g].skew(axis = 1)
#         df['c_sum'] = df[features_c].sum(axis = 1)
#         df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
#         df['c_kurt'] = df[features_c].kurtosis(axis = 1)
#         df['c_skew'] = df[features_c].skew(axis = 1)
#         df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
#         df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
#         df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
        df['c52_c42'] = df['c-52'] * df['c-42']
        df['c13_c73'] = df['c-13'] * df['c-73']
        df['c26_c13'] = df['c-23'] * df['c-13']
        df['c33_c6'] = df['c-33'] * df['c-6']
        df['c11_c55'] = df['c-11'] * df['c-55']
        df['c38_c63'] = df['c-38'] * df['c-63']
        df['c38_c94'] = df['c-38'] * df['c-94']
        df['c13_c94'] = df['c-13'] * df['c-94']
        df['c4_c52'] = df['c-4'] * df['c-52']
        df['c4_c42'] = df['c-4'] * df['c-42']
        df['c13_c38'] = df['c-13'] * df['c-38']
        df['c55_c2'] = df['c-55'] * df['c-2']
        df['c55_c4'] = df['c-55'] * df['c-4']
        df['c4_c13'] = df['c-4'] * df['c-13']
        df['c82_c42'] = df['c-82'] * df['c-42']
        df['c66_c42'] = df['c-66'] * df['c-42']
        df['c6_c38'] = df['c-6'] * df['c-38']
        df['c2_c13'] = df['c-2'] * df['c-13']
        df['c62_c42'] = df['c-62'] * df['c-42']
        df['c90_c55'] = df['c-90'] * df['c-55']
        
        
        for feature in features_c:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
        for feature in gsquarecols:
            df[f'{feature}_squared'] = df[feature] ** 2        
        
    return train, test

train_features2,test_features2=fe_stats(train_features2,test_features2)


train_features_stats=train_features2.iloc[:,902:]
test_features_stats=test_features2.iloc[:,902:]


train_features = pd.concat((train_features, train_features_cluster,train_cluster_pca,train_features_stats), axis=1)
test_features = pd.concat((test_features, test_features_cluster,test_cluster_pca,test_features_stats), axis=1)


train = train_features
train = train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test=test_features

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

# train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
# test = pd.get_dummies(test, columns=['cp_time','cp_dose'])
train['cp_time'] = train['cp_time'].map({24: 0, 48: 0.5, 72: 1})
test['cp_time'] = test['cp_time'].map({24: 0, 48: 0.5, 72: 1})
train["cp_dose"] = (train["cp_dose"]=="D1") + 0
test["cp_dose"] = (test["cp_dose"]=="D1") + 0

In [ ]:
feature_cols = [c for c in train.columns ]
feature_cols = [c for c in feature_cols if c not in ['sig_id']]


train.drop(columns=["sig_id"], inplace=True)
test.drop(columns=["sig_id"], inplace=True)
train_targets_scored.drop(columns=["sig_id"], inplace=True)
X_test = test.values

In [ ]:
MAX_EPOCH=200

In [ ]:
train_targets_scored=train_targets_scored[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

res=train_targets_scored.copy()
ss=submission.copy()

In [ ]:
N_STARTS = 3

res.loc[:, train_targets_scored.columns] = 0
ss.loc[:, train_targets_scored.columns] = 0
NB_SPLITS = 5

scores_auc = []
for seed in range(N_STARTS):
    #seed_everything(seed)
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=(seed+10), shuffle=True)
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, train_targets_scored)):
        print("FOLDS : ", fold_nb)
        
        ## model
#         X_train, y_train = train.values[train_idx, :], train_targets_scored.values[train_idx, :]
#         X_val, y_val = train.values[val_idx, :], train_targets_scored.values[val_idx, :]
#         tabnet_params = dict(n_d=24, n_a=40, n_steps=1, gamma=1.3,
#                      lambda_sparse=0, optimizer_fn=torch.optim.Adam,
#                      optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
#                      mask_type='entmax',
#                      scheduler_params=dict(mode="min",
#                                            patience=5,
#                                            min_lr=1e-5,
#                                            factor=0.9,),
#                      scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
#                      verbose=10,
#                      seed=seed
#                      )
#         model = TabNetRegressor(**tabnet_params)

#         model.fit(X_train=X_train,
#                   y_train=y_train,
#                   eval_set=[(X_val, y_val)],
#                   eval_name = ["val"],
#                   eval_metric = ["logits_ll"],
#                   max_epochs=MAX_EPOCH,
#                   patience=20, batch_size=1024, virtual_batch_size=128,
#                   num_workers=1, drop_last=False,
#                   # use binary cross entropy as this is not a regression problem
#                   loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
#         model.save_model("tabnet_model"+f"_fold_{fold_nb}_seed_{seed}")
        model = TabNetRegressor()
        model.load_model(f"../input/tabnetregressor-try-na-train-infe/tabnet_model_fold_{fold_nb}_seed_{seed}.zip")
        
#         preds_val = model.predict(X_val)
#         # Apply sigmoid to the predictions
#         preds =  1 / (1 + np.exp(-preds_val))
# #         score = np.min(model.history["val_logits_ll"])

#         ## save oof to compute the CV later
#         res.loc[val_idx, train_targets_scored.columns] += preds 
        

        # preds on test
        preds_test = model.predict(X_test)
        test_preds = (1 / (1 + np.exp(-preds_test)))
        ss.loc[:, train_targets_scored.columns] += test_preds 
# oof_preds_all = np.concatenate(oof_preds)
# oof_targets_all = np.concatenate(oof_targets)
# test_preds_all = np.stack(test_cv_preds)
# res.loc[:, train_targets_scored.columns] /= N_STARTS
ss.loc[:, train_targets_scored.columns] /= ((fold_nb+1) * N_STARTS)

In [ ]:
target_cols=train_targets_scored.columns
target_cols

In [ ]:
train_targets_scored0 = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test1= pd.read_csv('../input/lish-moa/test_features.csv')
train1= pd.read_csv('../input/lish-moa/train_features.csv')

In [ ]:
ss.loc[test1['cp_type']=="ctl_vehicle", target_cols] = 0

In [ ]:
# ss

In [ ]:
# tmp1=pd.read_csv("../input/tabnetregressor-try-na-train-infe/submission.csv")

In [ ]:
# tmp1

In [ ]:
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #6.946e-06

In [ ]:
ss_model9=ss

In [ ]:
gc.collect()

# Model10

In [ ]:
SEED=42
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(SEED)

In [ ]:
data_path = "../input/lish-moa/"
train = pd.read_csv(data_path+'train_features.csv')
train.drop(columns=["sig_id"], inplace=True)

train_targets_scored = pd.read_csv(data_path+'train_targets_scored.csv')
train_targets_scored.drop(columns=["sig_id"], inplace=True)

test = pd.read_csv(data_path+'test_features.csv')
test.drop(columns=["sig_id"], inplace=True)

submission = pd.read_csv(data_path+'sample_submission.csv')

remove_vehicle = False

if remove_vehicle:
    kept_index = train['cp_type']=='trt_cp'
    train = train.loc[kept_index].reset_index(drop=True)
    train_targets_scored = train_targets_scored.loc[kept_index].reset_index(drop=True)

train["cp_type"] = (train["cp_type"]=="trt_cp") + 0
train["cp_dose"] = (train["cp_dose"]=="D1") + 0

test["cp_type"] = (test["cp_type"]=="trt_cp") + 0
test["cp_dose"] = (test["cp_dose"]=="D1") + 0

# X_test = test.values

c_cols = train.columns[train.columns.str.startswith('c-')]
g_cols = train.columns[train.columns.str.startswith('g-')]

In [ ]:
def fe_stats(train, test):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    for df in [train, test]:
#         df['g_sum'] = df[features_g].sum(axis = 1)
#         df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
#         df['g_kurt'] = df[features_g].kurtosis(axis = 1)
#         df['g_skew'] = df[features_g].skew(axis = 1)
#         df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
#         df['c_std'] = df[features_c].std(axis = 1)
#         df["gstd_div_cmean"]=df[features_g].std(axis = 1)/df[features_c].mean(axis = 1)
#         df['c_kurt'] = df[features_c].kurtosis(axis = 1)
#         df['c_skew'] = df[features_c].skew(axis = 1)
#         df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
#         df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
#         df['gc_std'] = df[features_g + features_c].std(axis = 1)
#         df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
#         df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
    return train, test

train, test = fe_stats(train, test)
X_test = test.values

In [ ]:
MAX_EPOCH=200

In [ ]:
N_STARTS = 3

# scores_auc_all= []

# test_cv_preds = []
# res.loc[:, train_targets_scored.columns] = 0
ss.loc[:, train_targets_scored.columns] = 0
NB_SPLITS = 5
#mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
# oof_preds = []
# oof_targets = []
# scores = []
scores_auc = []
for seed in range(N_STARTS):
    #seed_everything(seed)
#     mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=(seed+20), shuffle=True)
    for fold_nb in range(NB_SPLITS):
        print("FOLDS : ", fold_nb)
        
        ## model
#         X_train, y_train = train.values[train_idx, :], train_targets_scored.values[train_idx, :]
#         X_val, y_val = train.values[val_idx, :], train_targets_scored.values[val_idx, :]
# #         tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
#                      lambda_sparse=0, optimizer_fn=torch.optim.Adam,
#                      optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
#                      mask_type='entmax',
#                      scheduler_params=dict(mode="min",
#                                            patience=5,
#                                            min_lr=1e-5,
#                                            factor=0.9,),
#                      scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
#                      verbose=10,
#                      seed=seed
#                      )
#         model = TabNetRegressor(**tabnet_params)

#         model.fit(X_train=X_train,
#                   y_train=y_train,
#                   eval_set=[(X_val, y_val)],
#                   eval_name = ["val"],
#                   eval_metric = ["logits_ll"],
#                   max_epochs=MAX_EPOCH,
#                   patience=20, batch_size=1024, virtual_batch_size=128,
#                   num_workers=1, drop_last=False,
#                   # use binary cross entropy as this is not a regression problem
#                   loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
#         model.save_model("tabnet_model"+f"_fold_{fold_nb}_seed_{seed}")
        model = TabNetRegressor()
        model.load_model(f"../input/tabnetregressor-2-0-train-infer-oof/tabnet_model_fold_{fold_nb}_seed_{seed}.zip")

#         preds_val = model.predict(X_val)
#         # Apply sigmoid to the predictions
#         preds =  1 / (1 + np.exp(-preds_val))

#     #     name = cfg.save_name + f"_fold{fold_nb}"
#     #     model.save_model(name)
#         ## save oof to compute the CV later
#         res.loc[val_idx, train_targets_scored.columns] += preds 
        
# #         oof_preds.append(preds)
#         oof_targets.append(y_val)
#         scores.append(score)

        # preds on test
        preds_test = model.predict(X_test)
        test_preds = (1 / (1 + np.exp(-preds_test)))
        ss.loc[:, train_targets_scored.columns] += test_preds 
# oof_preds_all = np.concatenate(oof_preds)
# oof_targets_all = np.concatenate(oof_targets)
# test_preds_all = np.stack(test_cv_preds)
# res.loc[:, train_targets_scored.columns] /= N_STARTS
ss.loc[:, train_targets_scored.columns] /= ((fold_nb+1) * N_STARTS)

In [ ]:
target_cols=train_targets_scored.columns
train_targets_scored0 = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test1= pd.read_csv('../input/lish-moa/test_features.csv')
train1= pd.read_csv('../input/lish-moa/train_features.csv')
ss.loc[test1['cp_type']=="ctl_vehicle", target_cols] = 0
# ss

In [ ]:
# tmp1=pd.read_csv("../input/tabnetregressor-2-0-train-infer-oof/submission.csv")
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #1.9964e-06

In [ ]:
ss_model10=ss
gc.collect()

# Model11

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
c_cols = train_features.columns[train_features.columns.str.startswith('c-')]
g_cols = train_features.columns[train_features.columns.str.startswith('g-')]
cont_cols = g_cols.to_list() + c_cols.to_list()

In [ ]:
#RankGauss

for col in (list(c_cols) + list(g_cols)):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
    
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)




In [ ]:
def fe_pca(train, test, n_components_g = 28, n_components_c = 5, SEED = 123):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    def create_pca(train, test, features, kind = 'g', n_components = n_components_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        pca = PCA(n_components = n_components,  random_state = SEED)
        if kind == 'g':
            #gpca = pca.fit(data)
            gpca = pickle.load(open("../input/moa-pytorch-feature-engineering-0-01846/gpca.pkl","rb"))
            data = gpca.transform(data)
            #pickle.dump(gpca, open('gpca.pkl', 'wb'))
        if kind == 'c':
            #cpca = pca.fit(data)
            cpca = pickle.load(open("../input/moa-pytorch-feature-engineering-0-01846/cpca.pkl","rb"))
            data = cpca.transform(data)
            #pickle.dump(cpca, open('cpca.pkl', 'wb'))
        
        columns = [f'pca_{kind}{i + 1}' for i in range(n_components)]
        data = pd.DataFrame(data, columns = columns)
        train_ = data.iloc[:train.shape[0]]
        test_ = data.iloc[train.shape[0]:].reset_index(drop = True)
        train = pd.concat([train, train_], axis = 1)
        test = pd.concat([test, test_], axis = 1)
        return train, test
    
    train, test = create_pca(train, test, features_g, kind = 'g', n_components = n_components_g)
    train, test = create_pca(train, test, features_c, kind = 'c', n_components = n_components_c)
    return train, test

train_features, test_features = fe_pca(train_features, test_features, n_components_g = 600, n_components_c = 50, SEED = 42)

train_features.shape

In [ ]:
# Function to extract common stats features
def fe_stats(train, test):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    for df in [train, test]:
#         df['g_sum'] = df[features_g].sum(axis = 1)
#        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
#         df['g_kurt'] = df[features_g].kurtosis(axis = 1)
#         df['g_skew'] = df[features_g].skew(axis = 1)
#         df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
#         df['c_std'] = df[features_c].std(axis = 1)
#         df["gstd_div_cmean"]=df[features_g].std(axis = 1)/df[features_c].mean(axis = 1)
#         df['c_kurt'] = df[features_c].kurtosis(axis = 1)
#         df['c_skew'] = df[features_c].skew(axis = 1)
#         df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
#         df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
#         df['gc_std'] = df[features_g + features_c].std(axis = 1)
#         df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
#         df['gc_skew'] = df[features_g + features_c].skew(axis = 1)

    return train.iloc[:,-2:], test.iloc[:,-2:]


train_add,test_add=fe_stats(train_features, test_features)

tmp=train_add.shape[1]
train_features=train_features.iloc[:,:-tmp]
test_features=test_features.iloc[:,:-tmp]

In [ ]:
# var_thresh = VarianceThreshold(0.8)  #<-- Update

# #data = train_features.append(test_features)
# #data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])
# var_thresh.fit(train_features.iloc[:, 4:])
# mask0=var_thresh.get_support(indices=True)+4
# pickle.dump(mask0, open('mask0.pkl', 'wb'))
mask0 = pickle.load(open("../input/moa-pytorch-feature-engineering-0-01846/mask0.pkl","rb"))

train_features_transformed=train_features[train_features.columns[mask0]]

test_features_transformed=test_features[train_features.columns[mask0]]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

In [ ]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 2, 'D2': 1})
#     df = df.join(pd.get_dummies(df['cp_time'], drop_first=False, prefix='cp_time'))
#     df = df.drop('cp_time', axis=1)
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 1, 48: 2, 72: 3})
    df.loc[df.cp_type==1,'cp_dose']=0
    #del df['sig_id']
    return df

   
train = preprocess(train_features)
test = preprocess(test_features)


#del train_targets_scored['sig_id']

# train_targets = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
# train = train.loc[train['cp_type']==0].reset_index(drop=True)

del train['cp_type']
del test['cp_type']

train = pd.concat([train, train_add], axis = 1)
test = pd.concat([test, test_add], axis = 1)
train

In [ ]:
train = train.merge(train_targets_scored, on='sig_id')


target=train[train_targets_scored.columns]

SEED = 42
FOLDS = 5


targets=target.columns[1:]
scored = target.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)
    
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=5)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
# folds

In [ ]:
# print(train.shape)
# print(folds.shape)
# print(test.shape)
# print(target.shape)
# print(sample_submission.shape)

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds


class Model(nn.Module):      # <-- Update
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.4)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.4)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)


In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 8e-6
NFOLDS = 5            #<-- Update
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
    
#     model.to(DEVICE)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
#     loss_fn = nn.BCEWithLogitsLoss()
    
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
#     best_loss = np.inf
    
#     for epoch in range(EPOCHS):
        
#         train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
#         print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#         valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#         print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
#         if valid_loss < best_loss:
            
#             best_loss = valid_loss
#             oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), f"SEED_{seed}_FOLD_{fold}.pth")
        
#         elif(EARLY_STOP == True):
            
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
   
    model.load_state_dict(torch.load(f"../input/moa-pytorch-feature-engineering-0-01846/SEED_{seed}_FOLD_{fold}.pth","cpu"))
    model.to(DEVICE)
    print(f"../input/moa-pytorch-feature-engineering-0-01846/SEED_{seed}_FOLD_{fold}.pth")
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed):
  
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS

    return predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [0, 1, 2]  #<-- Update

predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    predictions_ = run_k_fold(NFOLDS, seed)

    predictions += predictions_ / len(SEED)

test[target_cols] = predictions

In [ ]:
len(target_cols)

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.loc[test_features['cp_type']=="ctl_vehicle", target_cols] = 0
# sub

In [ ]:
# tmp1=pd.read_csv("../input/moa-pytorch-feature-engineering-0-01846/submission.csv")
# (abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #2.0464e-06

In [ ]:
ss_model11=sub
gc.collect()

# Model12


In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

c_cols = train_features.columns[train_features.columns.str.startswith('c-')]
g_cols = train_features.columns[train_features.columns.str.startswith('g-')]
cont_cols = g_cols.to_list() + c_cols.to_list()
ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(seed=42)



In [ ]:
#RankGauss

for col in (list(c_cols) + list(g_cols)):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
    
def fe_pca(train, test, n_components_g = 28, n_components_c = 5, SEED = 123):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    def create_pca(train, test, features, kind = 'g', n_components = n_components_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        pca = PCA(n_components = n_components,  random_state = SEED)
        if kind == 'g':
            #gpca = (pca.fit(data))
            gpca = pickle.load(open("../input/basic-nn-dummy-try-col/gpca.pkl","rb"))
            data = (gpca.transform(data))
            #pickle.dump(gpca, open('gpca.pkl', 'wb'))
        if kind == 'c':
            #cpca = (pca.fit(data))
            cpca = pickle.load(open("../input/basic-nn-dummy-try-col/cpca.pkl","rb"))
            data = (cpca.transform(data))
            #pickle.dump(cpca, open('cpca.pkl', 'wb'))
        
        columns = [f'pca_{kind}{i + 1}' for i in range(n_components)]
        data = pd.DataFrame(data, columns = columns)
        train_ = data.iloc[:train.shape[0]]
        test_ = data.iloc[train.shape[0]:].reset_index(drop = True)
        train = pd.concat([train, train_], axis = 1)
        test = pd.concat([test, test_], axis = 1)
        return train, test
    
    train, test = create_pca(train, test, features_g, kind = 'g', n_components = n_components_g)
    train, test = create_pca(train, test, features_c, kind = 'c', n_components = n_components_c)
    return train, test

train_features, test_features = fe_pca(train_features, test_features, n_components_g = 600, n_components_c = 50, SEED = 42)

# train_features

In [ ]:
# Function to extract common stats features
def fe_stats(train, test):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    for df in [train, test]:
#         df['g_sum'] = df[features_g].sum(axis = 1)
#        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
#         df['g_kurt'] = df[features_g].kurtosis(axis = 1)
#         df['g_skew'] = df[features_g].skew(axis = 1)
#         df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
#         df['c_std'] = df[features_c].std(axis = 1)
#         df["gstd_div_cmean"]=df[features_g].std(axis = 1)/df[features_c].mean(axis = 1)
#         df['c_kurt'] = df[features_c].kurtosis(axis = 1)
#         df['c_skew'] = df[features_c].skew(axis = 1)
#         df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
#         df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
#         df['gc_std'] = df[features_g + features_c].std(axis = 1)
#         df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
#         df['gc_skew'] = df[features_g + features_c].skew(axis = 1)

    return train.iloc[:,-2:], test.iloc[:,-2:]

train_add,test_add=fe_stats(train_features, test_features)

tmp=train_add.shape[1]
train_features=train_features.iloc[:,:-tmp]
test_features=test_features.iloc[:,:-tmp]

In [ ]:
var_thresh = VarianceThreshold(0.8)  #<-- Update

#data = train_features.append(test_features)
#data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])
# var_thresh.fit(train_features.iloc[:, 4:])
# mask0=var_thresh.get_support(indices=True)+4
# pickle.dump(mask0, open('mask0.pkl', 'wb'))
mask0 = pickle.load(open("../input/basic-nn-dummy-try-col/mask0.pkl","rb"))


train_features_transformed=train_features[train_features.columns[mask0]]

test_features_transformed=test_features[train_features.columns[mask0]]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

In [ ]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 2, 'D2': 1})
#     df = df.join(pd.get_dummies(df['cp_time'], drop_first=False, prefix='cp_time'))
#     df = df.drop('cp_time', axis=1)
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 1, 48: 2, 72: 3})
    df.loc[df.cp_type==1,'cp_dose']=0
    del df['sig_id']
    return df

   
train = preprocess(train_features)
test = preprocess(test_features)


del train_targets['sig_id']

del train['cp_type']
del test['cp_type']

def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

train=process_data(train)
test=process_data(test)

train = pd.concat([train, train_add], axis = 1)
test = pd.concat([test, test_add], axis = 1)

# train

In [ ]:
smooth=0.001
p_min = smooth
p_max = 1-smooth
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (num_columns, )),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.1),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="swish",kernel_initializer="glorot_normal")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="swish",kernel_initializer="glorot_normal")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="swish",kernel_initializer="glorot_normal")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid",kernel_initializer="glorot_normal"))
    ])
    #opt = tf.optimizers.Adam(learning_rate = 0.001)
    #opt = tfa.optimizers.SWA(opt)
    opt=tfa.optimizers.AdamW(weight_decay=1e-6,learning_rate=0.001)
    model.compile(opt,
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=smooth),metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'),logloss]
                  )
    return model

top_feats=list(range(len(train.columns)))



In [ ]:
bad_target=list(train_targets.columns[train_targets.sum(axis=0)<2])
good_target=list(train_targets.columns[train_targets.sum(axis=0)>500])

train_targets1=train_targets.drop(bad_target,axis=1)
train_targets1=train_targets1.drop(good_target,axis=1)

In [ ]:
N_STARTS = 3

res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=5,random_state=seed+40,shuffle=True).split(train.values[:, top_feats], train_targets1)):
        print(f'Fold {n}')

        model = create_model(len(top_feats))
#         reduce_lr_loss = ReduceLROnPlateau(monitor='val_logloss', factor=0.2, patience=2, mode='min', min_lr=1E-5)
#         sv = tf.keras.callbacks.ModelCheckpoint(
#             'seed-%i-fold-%i-model.h5'%(seed,n), monitor='val_loss', verbose=0, save_best_only=True,
#             save_weights_only=True, mode='min', save_freq='epoch')
        
#         model.fit(train.values[tr][:, top_feats],
#                   train_targets.astype(float).values[tr],shuffle=True,
#                   validation_data=(train.values[te][:, top_feats], train_targets.astype(float).values[te]),
#                   epochs=60, batch_size=128,
#                   callbacks=[early_stopping,reduce_lr_loss,sv], verbose=2
#                  )
        model.load_weights(f"../input/basic-nn-dummy-try-col/seed-{seed}-fold-{n}-model.h5")
        print(f"../input/basic-nn-dummy-try-col/seed-{seed}-fold-{n}-model.h5")
        ss.loc[:, train_targets.columns] += model.predict(test.values[:, top_feats])
#         res.loc[te, train_targets.columns] += model.predict(train.values[te][:, top_feats])
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
# res.loc[:, train_targets.columns] /= N_STARTS



In [ ]:
train1= pd.read_csv('../input/lish-moa/train_features.csv')

test1 = pd.read_csv('../input/lish-moa/test_features.csv')

train_targets0 = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

ss.loc[test1['cp_type']=="ctl_vehicle", train_targets.columns] = 0

In [ ]:
# ss

In [ ]:
# tmp1=pd.read_csv("../input/basic-nn-dummy-try-col-inferce/submission.csv")
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #2.980e-7

In [ ]:
ss_model12=ss
gc.collect()

# Model13


In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_nonscore = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

c_cols = train_features.columns[train_features.columns.str.startswith('c-')]
g_cols = train_features.columns[train_features.columns.str.startswith('g-')]
cont_cols = g_cols.to_list() + c_cols.to_list()
ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    
seed_everything(seed=42)

#RankGauss

for col in (list(c_cols) + list(g_cols)):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 2, 'D2': 1})
#     df = df.join(pd.get_dummies(df['cp_time'], drop_first=False, prefix='cp_time'))
#     df = df.drop('cp_time', axis=1)
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 1, 48: 2, 72: 3})
    df.loc[df.cp_type==1,'cp_dose']=0
    del df['sig_id']
    return df

def scaling(train, test):
    features = train.columns[2:]
    scaler = RobustScaler()
    scaler.fit(pd.concat([train[features], test[features]], axis = 0))
    train[features] = scaler.transform(train[features])
    test[features] = scaler.transform(test[features])
    return train, test


    
train = preprocess(train_features)
test = preprocess(test_features)


del train_targets['sig_id']
del train_nonscore['sig_id']

# train_targets = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
# train = train.loc[train['cp_type']==0].reset_index(drop=True)


tns=train_nonscore
tns2=tns.loc[:,tns.sum()>3]
# tns2

In [ ]:
train_targets1=pd.concat([train_targets,tns2],axis=1)

del train['cp_type']
del test['cp_type']

# Function to extract common stats features
def fe_stats(train, test):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    for df in [train, test]:
#         df['g_sum'] = df[features_g].sum(axis = 1)
#        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
#         df['g_kurt'] = df[features_g].kurtosis(axis = 1)
#         df['g_skew'] = df[features_g].skew(axis = 1)
#         df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
#         df['c_std'] = df[features_c].std(axis = 1)
#         df["gstd_div_cmean"]=df[features_g].std(axis = 1)/df[features_c].mean(axis = 1)
#         df['c_kurt'] = df[features_c].kurtosis(axis = 1)
#         df['c_skew'] = df[features_c].skew(axis = 1)
#         df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
#         df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
#         df['gc_std'] = df[features_g + features_c].std(axis = 1)
#         df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
#         df['gc_skew'] = df[features_g + features_c].skew(axis = 1)

    return train.iloc[:,-2:], test.iloc[:,-2:]

train_add,test_add=fe_stats(train, test)

tmp=train_add.shape[1]
train=train.iloc[:,:-tmp]
test=test.iloc[:,:-tmp]

# train

In [ ]:
def fe_pca(train, test, n_components_g = 28, n_components_c = 5, SEED = 123):
    
    features_g = list(g_cols)
    features_c = list(c_cols)
    
    def create_pca(train, test, features, kind = 'g', n_components = n_components_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        pca = PCA(n_components = n_components,  random_state = SEED)
        if kind == 'g':
            #gpca = pca.fit(data)
            gpca = pickle.load(open("../input/basic-nn-dummy-with-non-score/gpca.pkl","rb"))
            data = gpca.transform(data)
            #pickle.dump(gpca, open('gpca.pkl', 'wb'))
        if kind == 'c':
            #cpca = pca.fit(data)
            cpca = pickle.load(open("../input/basic-nn-dummy-with-non-score/cpca.pkl","rb"))
            data = cpca.transform(data)
            #pickle.dump(cpca, open('cpca.pkl', 'wb'))
            
        columns = [f'pca_{kind}{i + 1}' for i in range(n_components)]
        data = pd.DataFrame(data, columns = columns)
        train_ = data.iloc[:train.shape[0]]
        test_ = data.iloc[train.shape[0]:].reset_index(drop = True)
        train = pd.concat([train, train_], axis = 1)
        test = pd.concat([test, test_], axis = 1)
        return train, test
    
    train, test = create_pca(train, test, features_g, kind = 'g', n_components = n_components_g)
    train, test = create_pca(train, test, features_c, kind = 'c', n_components = n_components_c)
    return train, test
train, test = fe_pca(train, test, n_components_g = 280, n_components_c = 50, SEED = 123)

# train

In [ ]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(0.8)  #<-- Update
data = train.append(test)

# var_thresh.fit(data.iloc[:, 2:])
# mask0=var_thresh.get_support(indices=True)+2
# pickle.dump(mask0, open('mask0.pkl', 'wb'))
mask0 = pickle.load(open("../input/basic-nn-dummy-with-non-score/mask0.pkl","rb"))
data_transformed=data[data.columns[mask0]]

train_features_transformed = data_transformed[ : train.shape[0]]
test_features_transformed = data_transformed[-test.shape[0] : ]


train = pd.DataFrame(train[['cp_time','cp_dose']].values.reshape(-1, 2),\
                              columns=['cp_time','cp_dose'])

train = pd.concat([train, pd.DataFrame(train_features_transformed)], axis=1)


test = pd.DataFrame(test[['cp_time','cp_dose']].values.reshape(-1, 2),\
                             columns=['cp_time','cp_dose'])

test= pd.concat([test, pd.DataFrame(test_features_transformed)], axis=1)

train.shape

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

train=process_data(train)
test=process_data(test)

train = pd.concat([train, train_add], axis = 1)
test = pd.concat([test, test_add], axis = 1)

def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (num_columns, )),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="swish")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="swish")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tfa.layers.WeightNormalization(tf.keras.layers.Dense(532, activation="sigmoid"))
    ])
    #opt = tf.optimizers.Adam(learning_rate = 0.001)
    #opt = tfa.optimizers.SWA(opt)
    opt=tfa.optimizers.AdamW(weight_decay=1e-5)
    model.compile(opt,
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0005),metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc')]
                  )
    return model

len(list(train.columns))

In [ ]:
top_feats=list(range(len(train.columns)))

In [ ]:
N_STARTS = 3

res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=5,random_state=seed,shuffle=True).split(train.values[:, top_feats], train_targets)):
        print(f'Fold {n}')

        model = create_model(len(top_feats))
        #reduce_lr_loss = ReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=3, verbose=1, epsilon=0.0001, mode='max')
#         sv = tf.keras.callbacks.ModelCheckpoint(
#             'seed-%i-fold-%i-model.h5'%(seed,n), monitor='val_loss', verbose=0, save_best_only=True,
#             save_weights_only=True, mode='min', save_freq='epoch')
#         model.fit(train.values[tr][:, top_feats],
#                   train_targets1.astype(float).values[tr],shuffle=True,
#                   validation_data=(train.values[te][:, top_feats], train_targets1.astype(float).values[te]),
#                   epochs=60, batch_size=128,
#                   callbacks=[early_stopping,get_lr_callback(128),sv], verbose=2
#                  )
        model.load_weights(f"../input/basic-nn-dummy-with-non-score/seed-{seed}-fold-{n}-model.h5")
        print(f"../input/basic-nn-dummy-with-non-score/seed-{seed}-fold-{n}-model.h5")
        ss.loc[:, train_targets.columns] += model.predict(test.values[:, top_feats])[:,:206]
#         res.loc[te, train_targets.columns] += model.predict(train.values[te][:, top_feats])[:,:206]
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
# res.loc[:, train_targets.columns] /= N_STARTS

# metrics = []
# for _target in train_targets.columns:
#     metrics.append(log_loss(train_targets.loc[:, _target], res.loc[:, _target]))

In [ ]:
train1= pd.read_csv('../input/lish-moa/train_features.csv')

test1 = pd.read_csv('../input/lish-moa/test_features.csv')

train_targets0 = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
ss.loc[test1['cp_type']=="ctl_vehicle", train_targets.columns] = 0

In [ ]:
# ss

In [ ]:
# tmp1=pd.read_csv("../input/basic-nn-dummy-with-non-score-inferce/submission.csv")
# (abs(ss.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #2.9964e-7

In [ ]:
ss_model13=ss
gc.collect()

# Model14

In [ ]:
data_dir = '../input/lish-moa/'
os.listdir(data_dir)
train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')



In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

for col in (GENES + CELLS):
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
SEED_VALUE = 42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED_VALUE)

In [ ]:
n_comp = 600

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])

# gpca= (PCA(n_components=n_comp, random_state=SEED_VALUE).fit(data[GENES]))
# pickle.dump(gpca, open('gpca.pkl', 'wb'))
gpca=pickle.load(open("../input/pytorch-transfer-learning-with-k-folds-by-drug/gpca.pkl","rb"))

data2 = gpca.transform(data[GENES])

train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
# CELLS
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
# cpca = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit(data[CELLS]))
# pickle.dump(cpca, open('cpca.pkl', 'wb'))
cpca=pickle.load(open("../input/pytorch-transfer-learning-with-k-folds-by-drug/cpca.pkl","rb"))

data2 = cpca.transform(data[CELLS])
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
var_thresh = VarianceThreshold(0.8)
data = train_features.append(test_features)
# var_thresh.fit(data.iloc[:, 4:])
# mask0=var_thresh.get_support(indices=True)+4
# pickle.dump(mask0, open('mask0.pkl', 'wb'))
mask0 = pickle.load(open("../input/pytorch-transfer-learning-with-k-folds-by-drug/mask0.pkl","rb"))

data_transformed = data[data.columns[mask0]]

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)

test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)


In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
    
    

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)
num_features

In [ ]:

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128

WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

model = Model(num_features, num_all_targets)

In [ ]:

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in range(SEEDS):
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = 3
NFOLDS = 5
DRUG_THRESH = 18

train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
# train.head()

In [ ]:
def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
#     def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
#         x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
#         x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
#         train_dataset = MoADataset(x_train, y_train)
#         valid_dataset = MoADataset(x_valid, y_valid)

#         trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#         validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
#         optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
#         scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
#                                                   steps_per_epoch=len(trainloader),
#                                                   pct_start=PCT_START,
#                                                   div_factor=DIV_FACTOR[tag_name], 
#                                                   max_lr=MAX_LR[tag_name],
#                                                   epochs=EPOCHS)
        
#         loss_fn = nn.BCEWithLogitsLoss()
#         loss_tr = SmoothBCEwLogits(smoothing=0.001)

#         oof = np.zeros((len(train), len(target_cols_now)))
#         best_loss = np.inf
        
#         for epoch in range(EPOCHS):
#             if fine_tune_scheduler is not None:
#                 fine_tune_scheduler.step(epoch, model)

#             train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
#             valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
#             print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

#             if np.isnan(valid_loss):
#                 break
            
#             if valid_loss < best_loss:
#                 best_loss = valid_loss
#                 oof[val_idx] = valid_preds
#                 torch.save(model.state_dict(), f"{tag_name}_SEED_{seed_id}_FOLD{fold_id}_.pth")

#         return oof

#     fine_tune_scheduler = FineTuneScheduler(EPOCHS)

#     pretrained_model = Model(num_features, num_all_targets)
#     pretrained_model.to(DEVICE)

#     # Train on scored + nonscored targets
#     train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

#     # Load the pretrained model with the best loss
#     pretrained_model = Model(num_features, num_all_targets)
#     pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_SEED_{seed_id}_FOLD{fold_id}_.pth"))
#     pretrained_model.to(DEVICE)

#     # Copy model without the top layer
#     final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

#     # Fine-tune the model on scored targets only
#     oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    model.load_state_dict(torch.load(f"../input/pytorch-transfer-learning-with-k-folds-by-drug/SCORED_ONLY_SEED_{seed_id}_FOLD{fold_id}_.pth","cpu"))
    print(f"../input/pytorch-transfer-learning-with-k-folds-by-drug/SCORED_ONLY_SEED_{seed_id}_FOLD{fold_id}_.pth")
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return predictions

In [ ]:
def run_k_fold(NFOLDS, seed_id):

    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        pred_ = run_training(fold_id, seed_id)
        predictions += pred_ / NFOLDS
       
        
    return predictions

In [ ]:
SEED = [0, 1, 2]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))


for seed_id in SEED:
    predictions_ = run_k_fold(NFOLDS, seed_id)

    predictions += predictions_ / len(SEED)

test[target_cols] = predictions

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
# sub

In [ ]:
# tmp1=pd.read_csv("../input/pytorch-transfer-learning-with-k-folds-by-drug/submission.csv")
# (abs(sub.iloc[:,1:].values-tmp1.iloc[:,1:].values)).max() #9.623e-8

In [ ]:
ss_model14=sub
gc.collect()

# Ensemble

In [ ]:

ss_tot=[ss_model1,ss_model2,ss_model3,ss_model4,ss_model5,ss_model8,ss_model9,ss_model10,ss_model11,ss_model12,ss_model13,ss_model14]

In [ ]:
# for i in range(len(ss_tot)):
#     print(ss_tot[i].shape)

In [ ]:
ss_ensemble=ss_tot[0].copy()
ss_ensemble.iloc[:,1:]=0
ss_ensemble1=ss_ensemble.copy()

In [ ]:
# ss_tot[0]

In [ ]:
# ss_ensemble

In [ ]:
col_m = pickle.load(open("../input/clip-try-col-method-ensemble-12-model/col_m.pkl","rb"))
col_w = pickle.load(open("../input/clip-try-col-method-ensemble-12-model/col_w.pkl","rb"))

In [ ]:

for col in range(ss_ensemble.shape[1]-1):
    md = ss_tot[col_m[col][0]].iloc[:,1+col]
    for i,k in enumerate(col_m[col][1:]):
        
        md=col_w[col][i]*ss_tot[k].iloc[:,1+col] + (1-col_w[col][i])*md
    ss_ensemble1.iloc[:,1+col]=md

In [ ]:
ss_ensemble1.iloc[:,1:].values.max()

In [ ]:
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
test1=pd.read_csv("../input/lish-moa/test_features.csv")
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
targets_col=sample_submission.columns[1:]


In [ ]:
#ss_ensemble1

In [ ]:
ss_ensemble2=ss_ensemble.copy()

In [ ]:
for i in range(len(ss_tot)):
    ss_ensemble2.iloc[:,1:]+=ss_tot[i].iloc[:,1:]/len(ss_tot)

In [ ]:
ss_ensemble.iloc[:,1:]=ss_ensemble1.iloc[:,1:]*0.7+ss_ensemble2.iloc[:,1:]*0.3

In [ ]:
ss_ensemble.iloc[:,1:]=ss_ensemble.iloc[:,1:].clip(0.0005,0.9995)

In [ ]:

ss_ensemble.loc[test1['cp_type']=="ctl_vehicle", targets_col] = 0

In [ ]:
ss_ensemble.to_csv('submission.csv', index=False)